### Movie Mania
#### A Project to analyze the relationship between a movie’s sentiment on Social Media and its crowd-sourced rating.

In [4]:
#Import libraries
import sys
import time
import datetime
import json
import pandas as pd

from tweepy import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API

In [5]:
#Fill in twitter dev info.
consumer_key = '6UhBB4yCNDEnPcWrlmW84VVfK'
consumer_secret = 'F0ZVLxQXP4Pm0hBxYhur4MndlgxET7IUStYEbj0zy212Lo4TNJ'
access_token ='44631704-O9jj3LhyYewKzIXGes65WRkNhG6X5sLek8ZlVMDQF'
access_token_secret = 'y8nDFRPYqcMDqicOfEfm2C0vGJyPJ6g5NTl9IrNz5jZhX'

#This handles Twitter authetification and the connection to Twitter Streaming API
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)

class StreamParser(StreamListener):
    """ Controls how streaming data is parsed. Pass an outfile, or data will be writting to 
    sys.stdout (eg the screen)
    """
    def __init__(self, outfile=None, max_tweets=5, max_seconds=30):
        self.counter = 0
        self.start_time = time.time()
        # Set upper limits on maximum tweets or seconds before timeout
        self.max_tweets = max_tweets
        self.max_seconds = max_seconds
        if outfile:
            self.stdout = open(outfile, 'w')
        else:
            self.stdout = sys.stdout
    
    def on_data(self, data):
        """ Data is a string, but formatted for json. Parses it"""
        self.counter += 1
        # time data is all timestamps.
        current_time = time.time()
        run_time = current_time - self.start_time
                
        # If we want to read time, easiest way is to convert from timestamp using datetime
        formatted_time = datetime.datetime.now()
            
        # Technically, might not be the best place to put kill statements, but works well enough
        if self.max_tweets:
            if self.counter > self.max_tweets:
                self._kill_stdout()
                raise SystemExit('Max tweets of %s exceeded.  Killing stream... see %s' \
                             % (self.max_tweets, self.stdout))
  
        if self.max_seconds:
            if run_time > self.max_seconds:
                self._kill_stdout()
                raise SystemExit('Max time of %s seconds exceeded.  Killing stream... see %s' \
                                 % (self.max_seconds, self.stdout))

        print ('Tweet %s at %s.\nEllapsed: %.2f seconds\n' %(self.counter, formatted_time, run_time))

        # Write to file
        self.stdout.write(data)
        return True

    def _kill_stdout(self):
        """ If self.stdout is a file, close it.  If sys.stdout, pass"""
        if self.stdout is not sys.stdout:
            self.stdout.close() 
    
    def on_error(self, status):
        print (status)

In [6]:
# Stream 10 tweets, no matter the time it takes!
listener = StreamParser(outfile='infx547d4.txt', max_tweets=10, max_seconds=None)
stream = Stream(auth, listener)

#This line filter Twitter Streams to capture data by the keywords: 'candy', 'pants', 'ruby'
stream.filter(track=['twitter', 'pants', 'movie'])

Tweet 1 at 2017-04-24 02:17:41.849554.
Ellapsed: 1.52 seconds

Tweet 2 at 2017-04-24 02:17:41.880806.
Ellapsed: 1.55 seconds

Tweet 3 at 2017-04-24 02:17:41.896415.
Ellapsed: 1.56 seconds

Tweet 4 at 2017-04-24 02:17:41.896415.
Ellapsed: 1.56 seconds

Tweet 5 at 2017-04-24 02:17:41.896415.
Ellapsed: 1.56 seconds

Tweet 6 at 2017-04-24 02:17:41.926984.
Ellapsed: 1.59 seconds

Tweet 7 at 2017-04-24 02:17:41.926984.
Ellapsed: 1.59 seconds

Tweet 8 at 2017-04-24 02:17:41.929985.
Ellapsed: 1.60 seconds

Tweet 9 at 2017-04-24 02:17:41.932987.
Ellapsed: 1.60 seconds

Tweet 10 at 2017-04-24 02:17:41.932987.
Ellapsed: 1.60 seconds



SystemExit: Max tweets of 10 exceeded.  Killing stream... see <_io.TextIOWrapper name='infx547d4.txt' mode='w' encoding='cp1252'>

To exit: use 'exit', 'quit', or Ctrl-D.


In [8]:
tweets = []
tweet_file = open('infx547d4.txt', "r")
for line in tweet_file:
    try:
        tweet = json.loads(line)
        tweets.append(tweet)
    except:
        continue

In [9]:
tw = pd.DataFrame(tweets)

In [10]:
tw.head(2)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,filter_level,geo,...,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,None,None,Mon Apr 24 09:17:52 +0000 2017,NaN,"{'user_mentions': [], 'hashtags': [], 'symbols...",NaN,0,False,low,None,...,None,False,0,False,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",ZEE Jaipur Literature Festival to transform Th...,1493025472391,False,"{'profile_link_color': '1DA1F2', 'profile_imag..."
1,None,None,Mon Apr 24 09:17:52 +0000 2017,NaN,"{'user_mentions': [{'indices': [3, 15], 'id_st...",NaN,0,False,low,None,...,None,NaN,0,False,"{'display_text_range': [0, 140], 'geo': None, ...","<a href=""http://twitter.com/download/iphone"" r...",RT @wonderpuizz: เพลง Palette ของไอยูฟีทจียงใน...,1493025472490,False,"{'profile_link_color': '7FDBB6', 'profile_imag..."


In [11]:
tw[["user","text","favorite_count", "retweet_count","source",]][0:2]

,user,text,favorite_count,retweet_count,source
0,"{'profile_link_color': '1DA1F2', 'profile_imag...",ZEE Jaipur Literature Festival to transform Th...,0,0,"<a href=""http://twitter.com"" rel=""nofollow"">Tw..."
1,"{'profile_link_color': '7FDBB6', 'profile_imag...",RT @wonderpuizz: เพลง Palette ของไอยูฟีทจียงใน...,0,0,"<a href=""http://twitter.com/download/iphone"" r..."


In [12]:
len(tw)

10